In [ ]:
# Arranco montando Drive para acceder a los archivos necesarios

root_path = 'drive/My Drive/Big Data/'
my_path = root_path + 'TP3/'

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Instalo Spark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark import SparkContext

# Creo contexto
sc = SparkContext("local", "MyProgram")

In [ ]:
from pyspark.streaming import StreamingContext

# Creo contexto stream y cantidad de segundos que dura la ventana temporal
ssc = StreamingContext(sc,15)

In [ ]:
# Creo checkpoint

ssc.checkpoint("buffer")
stream = ssc.textFileStream(my_path + 'input/')

In [ ]:
# Procesamiento del flujo - Transformaciones y acciones a ejecutar

mappedStream = stream.map(lambda t: t.split("\t"))
mappedStream = mappedStream.map(lambda t: (t[0], t[1], t[2], t[4]))

filteredStream = mappedStream.filter(lambda t: t[3] == 'Otro')

mappedStream = filteredStream.map(lambda t: (1, (int(t[1]), int(t[2]), 1)))

reducedStream = mappedStream.reduceByKey(lambda t1, t2: (t1[0] + t2[0], t1[1] + t2[1], t1[2] + t2[2]))

# Actualizo el DStream

def fUpdate(newValues, history):
  if(history == None):
    history = (0,0,0)
  if(newValues == None or newValues == []):
    newValues = (0,0,0)
    return history
  else:
    newValues = (sum(map(lambda t: t[0], newValues)), sum(map(lambda t: t[1], newValues)), sum(map(lambda t: t[2], newValues)))
  return (history[0] + newValues[0], history[1] + newValues[1], newValues[2] + history[2])

history = reducedStream.updateStateByKey(fUpdate)

def fmap(t):
  return (t[1][0] / t[1][2], t[1][1] / t[1][2])

history.map(fmap).pprint()

In [ ]:
# Creación y finalización del thread

ssc.start()
ssc.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored